# **Plotting the theoretical $w(\theta)$ (or the $C_\ell$) for a given cosmology and a set of redshift distributions**

This notebook plots the theoretical angular correlation function $w(\theta)$ (or the $C_\ell$) based on a given cosmology and redshift distributions.


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"
from utils_template import TemplateInitializer
from utils_data import RedshiftDistributions

dataset = "DESY6"
nz_flags = ["fid"]
cosmology_templates = ["mice"]

# dataset = "DESY6_COLA"
# nz_flags = ["mocks"]
# cosmology_templates = ["mice"]

# dataset = "DESIY1_LRG_EZ"
# nz_flags = ["mocks"]
# cosmology_templates = ["desifid"]

# galaxy_bias = {bin_z: 1.5 for bin_z in range(30)}
galaxy_bias = {
    0: 1.7194059331248859, 1: 1.7368699826967668, 2: 1.7607987455921172, 3: 1.7749736119076094, 4: 1.7890893456002912, 5: 1.8114501448727005, 6: 1.8348016666587896, 
    7: 1.8515802970886328, 8: 1.873264947801679, 9: 1.9006652965477289, 10: 1.9190656610157775, 11: 1.9428973273005532, 12: 1.9674681189203926, 13: 1.9795195105987338, 
    14: 2.0050216799863922, 15: 2.028297965445937, 16: 2.052626912773687, 17: 2.0717320374413197, 18: 2.095962063193566, 19: 2.1139027654616473, 20: 2.1348435772469583, 
    21: 2.167938433844372, 22: 2.184873738277954, 23: 2.206108802161772, 24: 2.2367901867134, 25: 2.26043081180072, 26: 2.283253883706429, 27: 2.30796832662005, 28: 2.337125769691101
}

colors = {
    ("DESY6", "fid", "mice"): "blue", 
    ("DESY6", "fid", "planck"): "red", 
    ("DESY6", "fid", "desifid"): "purple",
    ("DESY6", "clusteringz", "mice"): "green", 
    ("DESY6", "clusteringz", "planck"): "orange",
    ("DESY6_COLA", "mocks", "mice"): "brown", 
    ("DESIY1_LRG_EZ", "mocks", "desifid"): "cyan", 
}

nbins = {
    ("DESY6", "fid"): 6,
    ("DESY6","clusteringz"): 4,
    ("DESY6_COLA", "mocks"): 6,
    ("DESIY1_LRG_EZ", "mocks"): 29,
}

linestyle = {"y": "-", "n": "--"}

nbins_max = max(nbins[dataset, nz_flag] for nz_flag in nz_flags)
text_added = {bin_z: False for bin_z in range(nbins_max)}

# for stats in ["wtheta", "C_ell"]:
for stats in ["wtheta"]:

    fig, axs = plt.subplots(nbins_max, 1, figsize=(8, 2 * nbins_max), sharex=True)
    
    for cosmology_template in cosmology_templates:
        for nz_flag in nz_flags:
            z_edges = RedshiftDistributions(dataset, nz_flag, verbose=False).z_edges
            color = colors[(dataset, nz_flag, cosmology_template)]
    
            if stats == "wtheta":
                wtheta = {}
                wtheta_bb = {}
                wtheta_bf = {}
                wtheta_ff = {}
            elif stats == "C_ell":
                C_ell = {}
                C_ell_bb = {}
                C_ell_bf = {}
                C_ell_ff = {}            
    
            for bin_z in range(nbins[dataset, nz_flag]):
                ax = axs[bin_z]
                
                for include_wiggles in ["y", "n"]:
                    
                    label = nz_flag + " " + cosmology_template + " (" + include_wiggles + ")"
                    
                    template_initializer = TemplateInitializer(
                        include_wiggles="" if include_wiggles == "y" else "_nowiggles",
                        dataset=dataset,
                        nz_flag=nz_flag,
                        cosmology_template=cosmology_template,
                        verbose=False,
                        base_path=None,
                    )
    
                    if stats == "wtheta":
                        wtheta_dict = template_initializer.load_wtheta(bin_z)
                        
                        theta = wtheta_dict["bb"][:, 0]
                        wtheta_bb[bin_z] = wtheta_dict["bb"][:, 1]
                        wtheta_bf[bin_z] = wtheta_dict["bf"][:, 1]
                        wtheta_ff[bin_z] = wtheta_dict["ff"][:, 1]
                        
                        wtheta[bin_z] = galaxy_bias[bin_z] ** 2 * wtheta_bb[bin_z] + galaxy_bias[bin_z] * wtheta_bf[bin_z] + wtheta_ff[bin_z]
                        
                        ax.semilogx(theta * 180 / np.pi, 100 * (theta * 180 / np.pi) * wtheta[bin_z], label=label, color=color, linestyle=linestyle[include_wiggles])
                    elif stats == "C_ell":
                        C_ell_dict = template_initializer.load_C_ell(bin_z)
                        
                        ell = C_ell_dict["bb"][:, 0]
                        C_ell_bb[bin_z] = C_ell_dict["bb"][:, 1]
                        C_ell_bf[bin_z] = C_ell_dict["bf"][:, 1]
                        C_ell_ff[bin_z] = C_ell_dict["ff"][:, 1]
                        
                        C_ell[bin_z] = galaxy_bias[bin_z] ** 2 * C_ell_bb[bin_z] + galaxy_bias[bin_z] * C_ell_bf[bin_z] + C_ell_ff[bin_z]
                        
                        ax.loglog(ell, ell * C_ell[bin_z], label=label, color=color, linestyle=linestyle[include_wiggles])
    
                z_edge = z_edges[bin_z]
                if stats == "wtheta":
                    if bin_z == nbins_max - 1:
                        ax.set_xlabel(r"$\theta$ (deg)", fontsize=22)
                    ax.set_ylabel(r"$10^2 \times \theta w(\theta)$", fontsize=22)
                    ax.set_xlim([10**-1, 180])
                elif stats == "C_ell":
                    if bin_z == nbins_max - 1:
                        ax.set_xlabel(r"$\ell$", fontsize=22)
                    ax.set_ylabel(r"$\ell C_\ell$", fontsize=22)
                    
                ax.tick_params(axis="x", labelsize=18)
                ax.tick_params(axis="y", labelsize=18)
    
                if stats == "wtheta":
                    xpos_text, ypos_text = 0.13, 0.1
                elif stats == "C_ell":
                    xpos_text, ypos_text = 0.13, 0.85
    
                if nbins[dataset, nz_flag] == nbins_max and not text_added[bin_z]:
                    if dataset not in ["DESIY1_LRG_Abacus", "DESIY1_LRG_EZ"]:
                        ax.text(xpos_text, ypos_text, f"{z_edge[0]} $< z <$ {z_edge[1]}", ha="center", va="center", transform=ax.transAxes, fontsize=18)
                    else:
                        ax.text(xpos_text, ypos_text, f"{z_edge[0]:.2f} $< z <$ {z_edge[1]:.2f}", ha="center", va="center", transform=ax.transAxes, fontsize=18)
                    text_added[bin_z] = True
    
            # if cosmology_template == cosmology_templates[0]:  # Add legend only to the first cosmology template
            axs[0].legend(loc="upper right", fontsize=18, ncol=len(nz_flags) * len(cosmology_templates))
    
    plt.tight_layout()
    plt.show()
